In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [2]:
from tra.models import *
from tra.test_runs_processing import *
from rep_portal.api import *
from functools import reduce
from django.contrib.auth.models import User

In [3]:
from tra.storage import get_storage_instance
from tra import utils

In [36]:
from django.db.models import Count
user = User.objects.get(username="plinta")

observed_test_insatnces = TestInstance.objects.filter(test_set__subscribers=user)
observed_test_runs = TestRun.objects.filter(test_instance__test_set__subscribers=user)

current_fb = FeatureBuild.objects.all().order_by("-name").first()
testruns_in_current_fb = observed_test_runs.filter(fb=current_fb)
na_testruns = testruns_in_current_fb.filter(result=TestRunResult.objects.get(name="not analyzed"))
envissue_testruns = testruns_in_current_fb.filter(result=TestRunResult.objects.get(name="environment issue"))
suspended_tis = observed_test_insatnces.filter(execution_suspended=True)
norun_tis = observed_test_insatnces.filter(no_run_in_rp=True)

top_na, top_na_count = na_testruns.values_list('fail_message').annotate(fm_count=Count('fail_message')).order_by('-fm_count').first()
top_envissue, top_envissue_count = envissue_testruns.values_list('comment').annotate(comment_count=Count('comment')).order_by('-comment_count').first()

response = {
    "env_issues": {
        "count": envissue_testruns.count(),
        "top": top_envissue,
        "top_count": top_envissue_count,
        "top_count_percent": int((top_envissue_count/envissue_testruns.count())*100)
    },
    "not_analyzed": {
        "count": na_testruns.count(),
        "top": top_na,
        "top_count": top_na_count,
        "top_count_percent": int((top_na_count/na_testruns.count())*100)
    },
    "test_instances":
    {
        "suspended": suspended_tis.count(),
        "no_run": norun_tis.count()
    }
}

In [37]:
response

{'env_issues': {'count': 88,
  'top': '[Env Issue Type: iphy] Analyzed by user autoanalyzer: MIB-SIB Acquisition Failed',
  'top_count': 68,
  'top_count_percent': 77},
 'not_analyzed': {'count': 5,
  'top': 'Number of messages 0 is not >= to 1',
  'top_count': 2,
  'top_count_percent': 40},
 'test_instances': {'suspended': 13, 'no_run': 0}}

2922